In [121]:
import pandas as pd
import re
from tqdm.notebook import tqdm
import time
import matplotlib.pyplot as plt
import numpy as np
from datetime import datetime

In [122]:
data = pd.read_csv('../data/fullsample.csv', nrows=10)
data

,JOBID,STATE,BEGIN,END,REQMEM,USEDMEM,REQTIME,USEDTIME,NODES,CPUS,PARTITION,EXITCODE
0,30616928,RUNNING,2021-07-31T22:15:00,Unknown,2048Mn,0,10:04:00,67-22:14:22,1,1,production,0:0
1,30853133,COMPLETED,2021-08-06T11:36:09,2021-09-05T11:36:32,262144Mn,20604.62M,30-00:00:00,30-00:00:23,1,1,cgw-platypus,0:0
2,30858137,COMPLETED,2021-08-06T19:04:39,2021-09-05T19:04:53,204800Mn,57553.77M,30-00:00:00,30-00:00:14,1,32,cgw-tbi01,0:0
3,30935078,COMPLETED,2021-08-09T16:52:51,2021-09-07T20:52:55,65536Mn,20577.96M,29-04:00:00,29-04:00:04,1,8,cgw-platypus,0:0
4,31364111_2,COMPLETED,2021-08-17T07:45:07,2021-09-10T16:45:24,16384Mn,9733.43M,24-09:00:00,24-09:00:17,1,1,production,0:0
5,31364111_3,COMPLETED,2021-08-17T07:45:07,2021-09-06T16:17:34,16384Mn,9708.04M,24-09:00:00,20-08:32:27,1,1,production,0:0
6,31364111_4,COMPLETED,2021-08-17T07:45:07,2021-09-06T06:25:11,16384Mn,9785.61M,24-09:00:00,19-22:40:04,1,1,production,0:0
7,31364111_5,COMPLETED,2021-08-17T07:45:07,2021-09-06T10:05:33,16384Mn,9966.38M,24-09:00:00,20-02:20:26,1,1,production,0:0
8,31364111_6,COMPLETED,2021-08-17T07:45:07,2021-09-05T12:53:04,16384Mn,9684.02M,24-09:00:00,19-05:07:57,1,1,production,0:0
9,31364111_9,COMPLETED,2021-08-17T07:45:07,2021-09-06T09:10:13,16384Mn,9802.24M,24-09:00:00,20-01:25:06,1,1,production,0:0


In [124]:
data = data.drop(data[data['END'] == 'Unknown'].index)
data.head()

,JOBID,STATE,BEGIN,END,REQMEM,USEDMEM,REQTIME,USEDTIME,NODES,CPUS,PARTITION,EXITCODE
1,30853133,COMPLETED,2021-08-06T11:36:09,2021-09-05T11:36:32,262144Mn,20604.62M,30-00:00:00,30-00:00:23,1,1,cgw-platypus,0:0
2,30858137,COMPLETED,2021-08-06T19:04:39,2021-09-05T19:04:53,204800Mn,57553.77M,30-00:00:00,30-00:00:14,1,32,cgw-tbi01,0:0
3,30935078,COMPLETED,2021-08-09T16:52:51,2021-09-07T20:52:55,65536Mn,20577.96M,29-04:00:00,29-04:00:04,1,8,cgw-platypus,0:0
4,31364111_2,COMPLETED,2021-08-17T07:45:07,2021-09-10T16:45:24,16384Mn,9733.43M,24-09:00:00,24-09:00:17,1,1,production,0:0
5,31364111_3,COMPLETED,2021-08-17T07:45:07,2021-09-06T16:17:34,16384Mn,9708.04M,24-09:00:00,20-08:32:27,1,1,production,0:0


In [125]:
data['BEGIN_DATE'] = pd.to_datetime(data['BEGIN']).dt.date
data['BEGIN_HOUR'] = pd.to_datetime(data['BEGIN']).dt.hour
#data['BEGIN_HOUR'] = data[data['BEGIN_HOUR'].strftime('%H')]

data['END_DATE'] = pd.to_datetime(data['END']).dt.date
data['END_HOUR'] = pd.to_datetime(data['END']).dt.hour

data = data[['JOBID', 'STATE', 'BEGIN_DATE', 'BEGIN_HOUR', 'END_DATE', 'END_HOUR', 'PARTITION', 'EXITCODE', 'REQMEM', 'USEDMEM', 'REQTIME', 'USEDTIME', 'NODES', 'CPUS']]

data

,JOBID,STATE,BEGIN_DATE,BEGIN_HOUR,END_DATE,END_HOUR,PARTITION,EXITCODE,REQMEM,USEDMEM,REQTIME,USEDTIME,NODES,CPUS
1,30853133,COMPLETED,2021-08-06,11,2021-09-05,11,cgw-platypus,0:0,262144Mn,20604.62M,30-00:00:00,30-00:00:23,1,1
2,30858137,COMPLETED,2021-08-06,19,2021-09-05,19,cgw-tbi01,0:0,204800Mn,57553.77M,30-00:00:00,30-00:00:14,1,32
3,30935078,COMPLETED,2021-08-09,16,2021-09-07,20,cgw-platypus,0:0,65536Mn,20577.96M,29-04:00:00,29-04:00:04,1,8
4,31364111_2,COMPLETED,2021-08-17,7,2021-09-10,16,production,0:0,16384Mn,9733.43M,24-09:00:00,24-09:00:17,1,1
5,31364111_3,COMPLETED,2021-08-17,7,2021-09-06,16,production,0:0,16384Mn,9708.04M,24-09:00:00,20-08:32:27,1,1
6,31364111_4,COMPLETED,2021-08-17,7,2021-09-06,6,production,0:0,16384Mn,9785.61M,24-09:00:00,19-22:40:04,1,1
7,31364111_5,COMPLETED,2021-08-17,7,2021-09-06,10,production,0:0,16384Mn,9966.38M,24-09:00:00,20-02:20:26,1,1
8,31364111_6,COMPLETED,2021-08-17,7,2021-09-05,12,production,0:0,16384Mn,9684.02M,24-09:00:00,19-05:07:57,1,1
9,31364111_9,COMPLETED,2021-08-17,7,2021-09-06,9,production,0:0,16384Mn,9802.24M,24-09:00:00,20-01:25:06,1,1
